# Intro ML SUP en BD

## Tarea 5

### Josue Arrieta Alfaro y Luis Felipe Cubero Vargas

Creación de la sesión Spark:

In [1]:
#import SparkSession
from pyspark.sql import SparkSession


In [93]:
# Crear el spark session object, llamarle "supervised_ml"
spark=SparkSession.builder.appName('supervised_ml').getOrCreate()


## Regression 

Carga de datos, archivo *Linear_regression_dataset.csv*:

In [67]:
# Carga de datos
df=spark.read.csv('Linear_regression_dataset.csv',inferSchema=True,header=True)



Se invocan las librerias correcpondientes a **LinearRegression**, asi como las de OneHotEncoder, StringIndexer, VectorAssembler:

In [11]:
# Importacion de libs y operaciones

from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler 

from pyspark.ml.regression import LinearRegression

Se visualizan algunos datos:

In [ ]:
print((df.count(), len(df.columns))) 

Se muestran los primeros 10 datos:

In [5]:
# primeros 10 datos


df.show(10)

+-----+-----+-----+-----+-----+-----+
|var_1|var_2|var_3|var_4|var_5|label|
+-----+-----+-----+-----+-----+-----+
|  734|  688|   81|0.328|0.259|0.418|
|  700|  600|   94| 0.32|0.247|0.389|
|  712|  705|   93|0.311|0.247|0.417|
|  734|  806|   69|0.315| 0.26|0.415|
|  613|  759|   61|0.302| 0.24|0.378|
|  748|  676|   85|0.318|0.255|0.422|
|  669|  588|   97|0.315|0.251|0.411|
|  667|  845|   68|0.324|0.251|0.381|
|  758|  890|   64| 0.33|0.274|0.436|
|  726|  670|   88|0.335|0.268|0.422|
+-----+-----+-----+-----+-----+-----+
only showing top 10 rows



## Feature Engineering

Creamos un solo vector con todos los features i.e 'var_1', 'var_2', 'var_3', 'var_4', 'var_5', a este le llamaremos "features" y como salida colocamos a 'label':

In [68]:
# Vector Ensamblador

df_assembler = VectorAssembler(inputCols=['var_1', 'var_2', 'var_3', 'var_4', 'var_5'], outputCol="features")
df = df_assembler.transform(df) 

In [69]:
# visulizacion de vector ensamblado compuesto por features y label

df.select(['features','label']).show(10,False)

+------------------------------+-----+
|features                      |label|
+------------------------------+-----+
|[734.0,688.0,81.0,0.328,0.259]|0.418|
|[700.0,600.0,94.0,0.32,0.247] |0.389|
|[712.0,705.0,93.0,0.311,0.247]|0.417|
|[734.0,806.0,69.0,0.315,0.26] |0.415|
|[613.0,759.0,61.0,0.302,0.24] |0.378|
|[748.0,676.0,85.0,0.318,0.255]|0.422|
|[669.0,588.0,97.0,0.315,0.251]|0.411|
|[667.0,845.0,68.0,0.324,0.251]|0.381|
|[758.0,890.0,64.0,0.33,0.274] |0.436|
|[726.0,670.0,88.0,0.335,0.268]|0.422|
+------------------------------+-----+
only showing top 10 rows



Partimos a continuación el set de datos en 75% training y 25% testing:

In [71]:
# Particion del data set
model_df=df.select(['features','label'])

train,test = model_df.randomSplit([0.75,0.25])

print(f"Size of train Dataset : {train.count()}" )
print(f"Size of test Dataset : {test.count()}" )

Size of train Dataset : 910
Size of test Dataset : 322


Creamos el Regresor Lineal: 

In [14]:
lr = LinearRegression()

Entrenamos el modelo de regresión lineal:

In [20]:
# Fit the model, le llamamos lr_model
lr_model=lr.fit(train)



Creamos el dataframe de prediciones (*predictions_df*) a partir del modelo de entrenamiento y el conjunto de datos test: 

In [21]:
predictions_df = lr_model.transform(test)

Visualizamos el contenido de *predictions_df*:

In [22]:
# visulizacion de predictions_df
predictions_df.show(10)


+--------------------+-----+-------------------+
|            features|label|         prediction|
+--------------------+-----+-------------------+
|[486.0,610.0,61.0...|0.332|0.31775435096321414|
|[498.0,672.0,61.0...|0.325| 0.3307007335423726|
|[511.0,576.0,76.0...|0.329|0.32793947736202866|
|[532.0,690.0,69.0...|0.351|0.33876835710366704|
|[541.0,830.0,60.0...| 0.33|0.34023928636925604|
|[543.0,747.0,60.0...|0.342|0.34237912968130146|
|[545.0,661.0,61.0...| 0.35|0.33903777407604496|
|[550.0,631.0,76.0...|0.318| 0.3361902069336389|
|[550.0,789.0,54.0...|0.359| 0.3428409510817033|
|[555.0,741.0,54.0...|0.348|  0.348443833978314|
+--------------------+-----+-------------------+
only showing top 10 rows



Ahora, evaluamos el modelo de Regresión Lineal, con los datos de TEST:

In [25]:
# evaluacion del modelo, le llamaremos model_predictions

model_predictions = lr_model.evaluate(test)


Imprimimos el valor de R2:

In [26]:
# valor de R2

model_predictions.r2


0.8427147591212798

Imprimimos el valor del meanSquaredError:

In [27]:
# valor del meanSquaredError

model_predictions.meanSquaredError


0.0001603537947150471

## Regresión con Árboles de Decisión

Importamos la librería *DecisionTreeRegressor*: 

In [46]:
# import lib

from pyspark.ml.regression import DecisionTreeRegressor

Creamos el Regresor DT, le llamaremos *dec_tree*:

In [48]:
# dec_tree

dec_tree = DecisionTreeRegressor()

Entrenamos el modelo:

In [49]:
# Train model, le llamaremos dec_tree_model

dec_tree_model = dec_tree.fit(train)


Cuánto es la profundidad máxima por defecto, de este algoritmo?

R/ La profundidad máxima es de 5 

Desplegamos las *featureImportances*:

In [53]:
dec_tree_model.featureImportances

SparseVector(5, {0: 0.972, 1: 0.0177, 2: 0.0021, 3: 0.0015, 4: 0.0068})

Evaluamos el modelo con los datos de entrenamiento:

In [56]:
# Make predictions, le llamaremos model_predictions 

model_predictions = dec_tree_model.transform(test)


In [57]:
# visualizamos

model_predictions.show()

+--------------------+-----+-------------------+
|            features|label|         prediction|
+--------------------+-----+-------------------+
|[464.0,640.0,66.0...|0.301|0.32072727272727286|
|[470.0,509.0,76.0...|0.319|0.32072727272727286|
|[473.0,499.0,73.0...|0.315|0.32072727272727286|
|[531.0,734.0,55.0...| 0.34| 0.3496666666666666|
|[536.0,531.0,83.0...|0.318|0.32072727272727286|
|[543.0,615.0,76.0...|0.333|0.32072727272727286|
|[543.0,747.0,60.0...|0.342| 0.3496666666666666|
|[548.0,636.0,63.0...|0.339| 0.3355714285714286|
|[548.0,770.0,60.0...|0.351| 0.3496666666666666|
|[556.0,675.0,67.0...|0.348| 0.3489999999999999|
|[567.0,595.0,74.0...|0.359| 0.3489999999999999|
|[570.0,655.0,66.0...| 0.34| 0.3489999999999999|
|[571.0,577.0,83.0...|0.368| 0.3489999999999999|
|[578.0,812.0,56.0...|0.351| 0.3568333333333334|
|[579.0,497.0,91.0...|0.352| 0.3489999999999999|
|[579.0,655.0,71.0...|0.357| 0.3489999999999999|
|[579.0,671.0,72.0...|0.338| 0.3489999999999999|
|[580.0,600.0,79.0..

Importamos el **RegressionEvaluator**

In [58]:
# import Evaluator

from pyspark.ml.evaluation import RegressionEvaluator



Usando *RegressionEvaluator* calculamos e imprimimos el valor de las metricas R2 y RMSE:

In [61]:
# R2 value of the model on test data 
dt_evaluator = RegressionEvaluator(metricName='r2')
dt_r2 = dt_evaluator.evaluate(model_predictions)
print(f'The r-square value of DecisionTreeRegressor is {dt_r2}')

# RMSE value of the model on test data 

dt_evaluator = RegressionEvaluator(metricName='rmse')
dt_rmse = dt_evaluator.evaluate(model_predictions)
print(f'The RMSE of DecisionTreeRegressor is {dt_rmse}')


The r-square value of DecisionTreeRegressor is 0.8345502811930378
The RMSE of DecisionTreeRegressor is 0.013415153794899626


## RandomForestRegressor

Importamos a *RandomForestRegressor*

In [62]:
# import lib

from pyspark.ml.regression import RandomForestRegressor



Creamos el Regresor RF:

In [73]:
# Regresor 

rf = RandomForestRegressor()


Entrenamos el modelo:

In [74]:
# Train model, le llamaremos rf_model
rf_model = rf.fit(train)


Desplegamos las *featureImportances*:

In [77]:
# importances 

rf_model.featureImportances


SparseVector(5, {0: 0.4714, 1: 0.0442, 2: 0.029, 3: 0.2605, 4: 0.1949})

Desplegamos el numero de arboles (Num of Trees)

In [80]:
# Numero de Trees

rf_model.getNumTrees


20

Evaluamos el modelo con los datos de entrenamiento, le llamaremos model_predictions:


In [82]:
# model_predictions

model_predictions = rf_model.transform(test)


Desplegamos los valores del *model_predictions*

In [83]:
model_predictions.show()

+--------------------+-----+-------------------+
|            features|label|         prediction|
+--------------------+-----+-------------------+
|[498.0,615.0,67.0...|0.318|0.32153336217828066|
|[498.0,672.0,61.0...|0.325|0.34202309305950573|
|[531.0,491.0,89.0...| 0.32|    0.3281007826835|
|[533.0,660.0,62.0...| 0.33|0.32700926114042966|
|[534.0,609.0,69.0...|0.329|0.32713378762965684|
|[548.0,636.0,63.0...|0.339| 0.3526177524123738|
|[556.0,674.0,62.0...|0.348| 0.3451152191607155|
|[558.0,688.0,67.0...| 0.35|0.34558683719785077|
|[562.0,546.0,79.0...| 0.35|0.34923071249711357|
|[569.0,544.0,82.0...|0.343|0.35047613325292015|
|[569.0,620.0,77.0...|0.349| 0.3551901455453773|
|[569.0,711.0,65.0...| 0.34|0.35073599923022186|
|[569.0,776.0,53.0...|0.348|0.35649328396355784|
|[570.0,578.0,82.0...|0.363|0.34701308143874393|
|[572.0,646.0,71.0...|0.329|0.34992489060168874|
|[573.0,656.0,75.0...|0.345| 0.3509083554751423|
|[575.0,680.0,68.0...|0.344|0.35346508216776545|
|[579.0,497.0,91.0..

Usando *RegressionEvaluator* calculamos e imprimimos el valor de las metricas R2 y RMSE:

In [84]:

# R2 value of the model on test data 
rf_evaluator = RegressionEvaluator(metricName='r2')
rf_r2 = rf_evaluator.evaluate(model_predictions)
print(f'The r-square value of DecisionTreeRegressor is {rf_r2}')

# RMSE value of the model on test data 

rf_evaluator = RegressionEvaluator(metricName='rmse')
rf_rmse = rf_evaluator.evaluate(model_predictions)
print(f'The RMSE of DecisionTreeRegressor is {rf_rmse}')





The r-square value of DecisionTreeRegressor is 0.8221586756997457
The RMSE of DecisionTreeRegressor is 0.01357132145938229


## Gradient-Boosted Tree Regressor

Importamos a GBTRegressor




In [90]:
# import

from pyspark.ml.regression import GBTRegressor


Creamos el Regresor GBTR, le llamaremos gbt:
 

In [94]:
# regresor
gbt= GBTRegressor()



Entrenamos el modelo:

In [95]:
# Train model, le llamaremos gbt_model
gbt_model = gbt.fit(train)


Desplegamos las featureImportances:

In [97]:
#Importances

gbt_model.featureImportances


SparseVector(5, {0: 0.2315, 1: 0.1996, 2: 0.2035, 3: 0.1884, 4: 0.177})

Evaluamos el modelo con los datos de entrenamiento, le llamaremos model_predictions:

In [101]:
# Model

model_predictions = gbt_model.transform(test)



Desplegamos los valores del *model_predictions*

In [102]:
# show 

model_predictions.show()


+--------------------+-----+-------------------+
|            features|label|         prediction|
+--------------------+-----+-------------------+
|[498.0,615.0,67.0...|0.318|0.31681420461709375|
|[498.0,672.0,61.0...|0.325| 0.3392567794630045|
|[531.0,491.0,89.0...| 0.32|0.32380886967899936|
|[533.0,660.0,62.0...| 0.33| 0.3283139223201472|
|[534.0,609.0,69.0...|0.329| 0.3127175150024977|
|[548.0,636.0,63.0...|0.339| 0.3446338255742373|
|[556.0,674.0,62.0...|0.348| 0.3491621799089955|
|[558.0,688.0,67.0...| 0.35| 0.3471152520670384|
|[562.0,546.0,79.0...| 0.35| 0.3446319857318228|
|[569.0,544.0,82.0...|0.343| 0.3413261466513804|
|[569.0,620.0,77.0...|0.349|0.35291202010287476|
|[569.0,711.0,65.0...| 0.34| 0.3440726951419147|
|[569.0,776.0,53.0...|0.348|0.35903247941609295|
|[570.0,578.0,82.0...|0.363|0.35213380332222677|
|[572.0,646.0,71.0...|0.329| 0.3394005155732629|
|[573.0,656.0,75.0...|0.345| 0.3392272043160616|
|[575.0,680.0,68.0...|0.344| 0.3477632190147875|
|[579.0,497.0,91.0..

Usando RegressionEvaluator calculamos e imprimimos el valor de las metricas R2 y RMSE:

In [103]:
#Select (prediction, true label) and compute test error

# R2 value of the model on test data 
gbt_evaluator = RegressionEvaluator(metricName='r2')
gbt_r2 = gbt_evaluator.evaluate(model_predictions)
print(f'The r-square value of DecisionTreeRegressor is {gbt_r2}')

# RMSE value of the model on test data 

gbt_evaluator = RegressionEvaluator(metricName='rmse')
gbt_rmse = gbt_evaluator.evaluate(model_predictions)
print(f'The RMSE of DecisionTreeRegressor is {gbt_rmse}')




The r-square value of DecisionTreeRegressor is 0.8310875103957545
The RMSE of DecisionTreeRegressor is 0.013226248473893955


 ## Exploracion de datos...

Usaremos el dataset https://archive.ics.uci.edu/ml/datasets/Bank+Marketing 

Indique a grandes razgos de que se trata este dataset:


Carga de datos, archivo bank_data.csv:


In [104]:
# Load csv Dataset 
df=spark.read.csv('bank_data.csv',inferSchema=True,header=True)

Determine la cantidad de datos en el dataset:

In [105]:
#number of records

print((df.count(),len(df.columns)))




(41188, 21)


A que dato corresponde cada columna?

In [107]:
# columns values

df.columns



['age',
 'job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'day_of_week',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'emp.var.rate',
 'cons.price.idx',
 'cons.conf.idx',
 'euribor3m',
 'nr.employed',
 'target_class']

Imprima el Schema:

In [106]:
#dataype of input data - Schema

df.printSchema()




root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- emp.var.rate: double (nullable = true)
 |-- cons.price.idx: double (nullable = true)
 |-- cons.conf.idx: double (nullable = true)
 |-- euribor3m: double (nullable = true)
 |-- nr.employed: double (nullable = true)
 |-- target_class: string (nullable = true)



En cuanto a la salida, como es la distrubución de clases?

In [111]:
# YES/NO Class Distribution

df.groupBy('target_class').count().show()




+------------+-----+
|target_class|count|
+------------+-----+
|          no|36548|
|         yes| 4640|
+------------+-----+



Una tarea típica, resulta de convertir los valores binarios en 1 y 0, usando como referencia "label", convierta los no/yes en 0/1:

In [112]:
from pyspark.sql import functions as F
from pyspark.sql import *

In [119]:
# Ingrese acá la instrucción: 

df = df.withColumn("target_class", F.when(F.col("target_class") == "yes", 1).otherwise(F.col("target_class")))
df = df.withColumn("target_class", F.when(F.col("target_class") == "no", 0).otherwise(F.col("target_class")))

In [120]:
# New 1/0 Class Distribution


df.groupBy('target_class').count().show()


+------------+-----+
|target_class|count|
+------------+-----+
|           0|36548|
|           1| 4640|
+------------+-----+



A continuación se presenta un ejercicio de Deep Learning para su revisión...

# Deep Learning 

Importamos las librerias necesarias:

In [121]:
import os
import numpy as np
import pandas as pd
from pyspark.sql.types import *
from pyspark.ml import Pipeline
from pyspark.sql import functions as f
from pyspark.sql.functions import udf, StringType
from pyspark.sql import SparkSession, functions as F
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.feature import OneHotEncoder, VectorAssembler, StringIndexer

Inicializamos la sesion SPARK:

In [122]:
spark = SparkSession.builder.appName('deep_learning').getOrCreate()

Leemos el dataset:

In [123]:
data = spark.read.csv('dl_data.csv', header=True, inferSchema=True)

In [124]:
data.printSchema()

root
 |-- Visit_Number_Bucket: string (nullable = true)
 |-- Page_Views_Normalized: double (nullable = true)
 |-- Orders_Normalized: integer (nullable = true)
 |-- Internal_Search_Successful_Normalized: double (nullable = true)
 |-- Internal_Search_Null_Normalized: double (nullable = true)
 |-- Email_Signup_Normalized: double (nullable = true)
 |-- Total_Seconds_Spent_Normalized: double (nullable = true)
 |-- Store_Locator_Search_Normalized: double (nullable = true)
 |-- Mapped_Last_Touch_Channel: string (nullable = true)
 |-- Mapped_Mobile_Device_Type: string (nullable = true)
 |-- Mapped_Browser_Type: string (nullable = true)
 |-- Mapped_Entry_Pages: string (nullable = true)
 |-- Mapped_Site_Section: string (nullable = true)
 |-- Mapped_Promo_Code: string (nullable = true)
 |-- Maped_Product_Name: string (nullable = true)
 |-- Mapped_Search_Term: string (nullable = true)
 |-- Mapped_Product_Collection: string (nullable = true)



Renombramos la columna TARGET:

In [125]:
data = data.withColumnRenamed('Orders_Normalized', 'label')

In [126]:
data.printSchema()

root
 |-- Visit_Number_Bucket: string (nullable = true)
 |-- Page_Views_Normalized: double (nullable = true)
 |-- label: integer (nullable = true)
 |-- Internal_Search_Successful_Normalized: double (nullable = true)
 |-- Internal_Search_Null_Normalized: double (nullable = true)
 |-- Email_Signup_Normalized: double (nullable = true)
 |-- Total_Seconds_Spent_Normalized: double (nullable = true)
 |-- Store_Locator_Search_Normalized: double (nullable = true)
 |-- Mapped_Last_Touch_Channel: string (nullable = true)
 |-- Mapped_Mobile_Device_Type: string (nullable = true)
 |-- Mapped_Browser_Type: string (nullable = true)
 |-- Mapped_Entry_Pages: string (nullable = true)
 |-- Mapped_Site_Section: string (nullable = true)
 |-- Mapped_Promo_Code: string (nullable = true)
 |-- Maped_Product_Name: string (nullable = true)
 |-- Mapped_Search_Term: string (nullable = true)
 |-- Mapped_Product_Collection: string (nullable = true)



Partimos lo datos en Train, Validation y Test:

In [127]:
train, validation, test  = data.randomSplit([0.7, 0.2, 0.1], 1234)

Construimos el Pipeline

In [128]:
categorical_columns = [item[0] for item in data.dtypes if item[1].startswith('string')]
numeric_columns = [item[0] for item in data.dtypes if item[1].startswith('double')]

indexers = [StringIndexer(inputCol=column, outputCol='{0}_index'.format(column)) for column in categorical_columns]

featuresCreator = VectorAssembler(inputCols=[indexer.getOutputCol() for indexer in indexers] + numeric_columns, outputCol="features")

layers = [len(featuresCreator.getInputCols()), 4, 2, 2]

classifier = MultilayerPerceptronClassifier(labelCol='label', featuresCol='features', maxIter=100, layers=layers, blockSize=128, seed=1234)

pipeline = Pipeline(stages=indexers + [featuresCreator, classifier])

Entrenamos...

In [129]:
model = pipeline.fit(train)

Validamos y Evaluamos

In [130]:
train_output_df = model.transform(train)
validation_output_df = model.transform(validation)
test_output_df = model.transform(test)

Llevamos a cabo, algunas predicciones:

In [131]:
train_predictionAndLabels = train_output_df.select("prediction", "label")
validation_predictionAndLabels = validation_output_df.select("prediction", "label")
test_predictionAndLabels = test_output_df.select("prediction", "label")

metrics = ['weightedPrecision', 'weightedRecall', 'accuracy']

for metric in metrics:
    evaluator = MulticlassClassificationEvaluator(metricName=metric)
    print('Train ' + metric + ' = ' + str(evaluator.evaluate(train_predictionAndLabels)))
    print('Validation ' + metric + ' = ' + str(evaluator.evaluate(validation_predictionAndLabels)))
    print('Test ' + metric + ' = ' + str(evaluator.evaluate(test_predictionAndLabels)))

Train weightedPrecision = 0.9678998185529217
Validation weightedPrecision = 0.9694550270413331
Test weightedPrecision = 0.9697940780082092
Train weightedRecall = 0.9673951151104359
Validation weightedRecall = 0.9689024390243902
Test weightedRecall = 0.9693974272173324
Train accuracy = 0.9673951151104359
Validation accuracy = 0.9689024390243902
Test accuracy = 0.9693974272173325


Puede mejorar el test accuracy del modelo variando alguno de los hyperparametros?